Modelo 3: LSTM + VAR

In [1]:
# Grid Search para o modelo LSTM + VAR

import numpy as np
import scipy.io as sio
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.api import VAR

# Definir os hiperparâmetros a serem testados
hyperparams_grid = {
    'sequence_length': [10, 20, 30],    # 3 opções
    'lstm_units': [64, 128],           # 2 opções
    'dropout_rate': [0.1, 0.2],         # 2 opções
    'learning_rate': [0.001, 0.0005],  # 2 opções
    'batch_size': [32, 64],             # 2 opções
    'maxlags_var': [5, 10]              # 2 opções
}

# Carregar os dados
file_path_temp_coefs = "temp_coefs.mat"
file_path_spatial_modes = "spatial_modes.mat"
file_path_mean_flow = "mean_flow.mat"
file_path_parameters = "parameters.mat"

mat_data_temp = sio.loadmat(file_path_temp_coefs)
coefs = mat_data_temp['coefs']
phi = sio.loadmat(file_path_spatial_modes)['phi']
y_positions = sio.loadmat(file_path_mean_flow)['y'].flatten()
Xavg = sio.loadmat(file_path_mean_flow)['Xavg'].flatten()
nmodos_pod = sio.loadmat(file_path_parameters)['nmodes']

# Definir o número de coeficientes a serem utilizados
num_coefs = 20  # Usando 20 coeficientes
print(f"Número total de modos utilizados: {num_coefs}")

# Separar os coeficientes em parte real e imaginária
real_coefs = np.real(coefs[:num_coefs, :])
imag_coefs = np.imag(coefs[:num_coefs, :])

# Divisão dos dados
first_interval_end = 4801
real_coefs_train = real_coefs[:, :first_interval_end:2]
imag_coefs_train = imag_coefs[:, :first_interval_end:2]
real_coefs_val = real_coefs[:, first_interval_end:]
imag_coefs_val = imag_coefs[:, first_interval_end:]

# Normalização
num_total_coefs = num_coefs * 2
scalers = []

coefs_train = np.vstack([real_coefs_train, imag_coefs_train])
coefs_val = np.vstack([real_coefs_val, imag_coefs_val])
coefs_train_normalized = np.zeros_like(coefs_train)
coefs_val_normalized = np.zeros_like(coefs_val)

for i in range(num_total_coefs):
    scaler_coef = MinMaxScaler(feature_range=(-1, 1))
    coefs_train_normalized[i, :] = scaler_coef.fit_transform(coefs_train[i, :].reshape(-1, 1)).flatten()
    coefs_val_normalized[i, :] = scaler_coef.transform(coefs_val[i, :].reshape(-1, 1)).flatten()
    scalers.append(scaler_coef)

def create_sequences(real_coefs_norm, imag_coefs_norm, seq_length):
    X, y = [], []
    num_samples = real_coefs_norm.shape[1] - seq_length
    for i in range(num_samples):
        X_seq_real = real_coefs_norm[:, i:i+seq_length]
        X_seq_imag = imag_coefs_norm[:, i:i+seq_length]
        X_seq = np.vstack([X_seq_real, X_seq_imag])
        X.append(X_seq)
        y_real = real_coefs_norm[:, i+seq_length]
        y_imag = imag_coefs_norm[:, i+seq_length]
        y_seq = np.hstack([y_real, y_imag])
        y.append(y_seq)
    X = np.array(X).transpose(0, 2, 1)
    y = np.array(y)
    return X, y


# Dicionário para armazenar os resultados
results = {}

from itertools import product

keys = hyperparams_grid.keys()
values = (hyperparams_grid[key] for key in keys)

total_combinations = np.prod([len(v) for v in hyperparams_grid.values()])
print(f"Total de combinações de hiperparâmetros: {total_combinations}")

combination_index = 1

for hp_values in product(*values):
    hyperparams = dict(zip(keys, hp_values))
    sequence_length = hyperparams['sequence_length']
    lstm_units = hyperparams['lstm_units']
    dropout_rate = hyperparams['dropout_rate']
    learning_rate = hyperparams['learning_rate']
    batch_size = hyperparams['batch_size']
    maxlags_var = hyperparams['maxlags_var']

    print(f"\nExecutando combinação {combination_index}/{total_combinations}: {hyperparams}")
    combination_index += 1

    # Criar sequências
    X_train, y_train = create_sequences(coefs_train_normalized[:num_coefs, :], coefs_train_normalized[num_coefs:, :], sequence_length)
    X_val, y_val = create_sequences(coefs_val_normalized[:num_coefs, :], coefs_val_normalized[num_coefs:, :], sequence_length)

    # Verificar se há dados suficientes
    if X_train.shape[0] == 0 or X_val.shape[0] == 0:
        print("Dados insuficientes para a sequência atual. Pulando para a próxima combinação.")
        continue

    # Definir o modelo LSTM
    model = Sequential()
    model.add(LSTM(lstm_units, input_shape=(X_train.shape[1], X_train.shape[2])))
    model.add(Dropout(dropout_rate))
    model.add(Dense(num_total_coefs))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Treinar o modelo
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_val, y_val),
                        callbacks=[early_stopping], verbose=0)

    # Previsões LSTM
    lstm_predictions_normalized = model.predict(X_val)
    lstm_predictions = np.zeros_like(lstm_predictions_normalized)
    y_val_original = np.zeros_like(y_val)
    for i in range(num_total_coefs):
        scaler = scalers[i]
        lstm_predictions[:, i] = scaler.inverse_transform(lstm_predictions_normalized[:, i].reshape(-1, 1)).flatten()
        y_val_original[:, i] = scaler.inverse_transform(y_val[:, i].reshape(-1, 1)).flatten()

    # Calcular resíduos
    residuals = y_val_original - lstm_predictions

    # Ajustar VAR nos resíduos
    model_var = VAR(residuals)
    results_aic = model_var.select_order(maxlags=maxlags_var)
    selected_lag = results_aic.aic if results_aic.aic is not None else maxlags_var
    model_var_fitted = model_var.fit(selected_lag)

    # Previsões VAR nos resíduos
    var_residuals_pred = model_var_fitted.fittedvalues
    var_residuals_pred = np.vstack([np.zeros((selected_lag, num_total_coefs)), var_residuals_pred])

    # Previsão final
    final_predictions = lstm_predictions[selected_lag:] + var_residuals_pred[selected_lag:]
    actual_values = y_val_original[selected_lag + sequence_length:]

    # Ajuste de dimensões
    min_length = min(final_predictions.shape[0], actual_values.shape[0])
    final_predictions = final_predictions[:min_length]
    actual_values = actual_values[:min_length]

    # Separar as partes real e imaginária
    pred_real = final_predictions[:, :num_coefs]
    pred_imag = final_predictions[:, num_coefs:]
    actual_real = actual_values[:, :num_coefs]
    actual_imag = actual_values[:, num_coefs:]

    # Cálculo do NMSE
    actual_modulus = np.sqrt(actual_real**2 + actual_imag**2)
    predicted_modulus = np.sqrt(pred_real**2 + pred_imag**2)

    nmse_modulus = []
    for c in range(num_coefs):
        mse = mean_squared_error(actual_modulus[:, c], predicted_modulus[:, c])
        variance = np.var(actual_modulus[:, c])
        nmse = mse / variance if variance != 0 else np.nan
        nmse_modulus.append(nmse)

    average_nmse_modulus = np.nanmean(nmse_modulus)

    # Salvar resultados
    result_key = f"SeqLen:{sequence_length}_Units:{lstm_units}_Dropout:{dropout_rate}_LR:{learning_rate}_Batch:{batch_size}_VARLag:{selected_lag}"
    results[result_key] = average_nmse_modulus
    print(f"NMSE médio do módulo dos coeficientes: {average_nmse_modulus:.6f}")

# Salvar os resultados em um arquivo
import pickle
with open('grid_search_results_lstm_var.pkl', 'wb') as f:
    pickle.dump(results, f)

# Encontrar a melhor combinação de hiperparâmetros
best_params = min(results, key=results.get)
print(f"\nMelhores hiperparâmetros: {best_params} com NMSE: {results[best_params]:.6f}")


Número total de modos utilizados: 20
Total de combinações de hiperparâmetros: 96

Executando combinação 1/96: {'sequence_length': 10, 'lstm_units': 64, 'dropout_rate': 0.1, 'learning_rate': 0.001, 'batch_size': 32, 'maxlags_var': 5}


c:\Users\FilipeAraujoXimenes\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
NMSE médio do módulo dos coeficientes: 1.912344

Executando combinação 2/96: {'sequence_length': 10, 'lstm_units': 64, 'dropout_rate': 0.1, 'learning_rate': 0.001, 'batch_size': 32, 'maxlags_var': 10}


c:\Users\FilipeAraujoXimenes\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
NMSE médio do módulo dos coeficientes: 1.914233

Executando combinação 3/96: {'sequence_length': 10, 'lstm_units': 64, 'dropout_rate': 0.1, 'learning_rate': 0.001, 'batch_size': 64, 'maxlags_var': 5}


...

175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step
NMSE médio do módulo dos coeficientes: 2.001674

Executando combinação 95/96: {'sequence_length': 30, 'lstm_units': 128, 'dropout_rate': 0.2, 'learning_rate': 0.0005, 'batch_size': 64, 'maxlags_var': 5}


c:\Users\FilipeAraujoXimenes\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


175/175 ━━━━━━━━━━━━━━━━━━━━ 3s 18ms/step
NMSE médio do módulo dos coeficientes: 2.001613

Executando combinação 96/96: {'sequence_length': 30, 'lstm_units': 128, 'dropout_rate': 0.2, 'learning_rate': 0.0005, 'batch_size': 64, 'maxlags_var': 10}


c:\Users\FilipeAraujoXimenes\anaconda3\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
NMSE médio do módulo dos coeficientes: 2.000933

Melhores hiperparâmetros: SeqLen:10_Units:64_Dropout:0.1_LR:0.001_Batch:32_VARLag:5 com NMSE: 1.912344


Modelo 4: LSTM + CNN + VAR

In [2]:
# Grid Search para o modelo LSTM + CNN + VAR

import numpy as np
import scipy.io as sio
import os
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Conv1D, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.api import VAR

hyperparams_grid = {
    'sequence_length': [10, 20, 30],          # 3 opções
    'lstm_units': [64, 128],                  # 2 opções
    'conv_filters': [[32, 16]],     # 2 opções
    'kernel_size': [3],                        # 1 opção
    'dropout_rate': [0.1, 0.2],                # 2 opções
    'learning_rate': [0.001, 0.0005],         # 2 opções
    'batch_size': [32, 64],                    # 2 opções
    'maxlags_var': [5, 10]                     # 2 opções
}

# Carregar os dados
file_path_temp_coefs = "temp_coefs.mat"
file_path_spatial_modes = "spatial_modes.mat"
file_path_mean_flow = "mean_flow.mat"
file_path_parameters = "parameters.mat"

mat_data_temp = sio.loadmat(file_path_temp_coefs)
coefs = mat_data_temp['coefs']
phi = sio.loadmat(file_path_spatial_modes)['phi']
y_positions = sio.loadmat(file_path_mean_flow)['y'].flatten()
Xavg = sio.loadmat(file_path_mean_flow)['Xavg'].flatten()
nmodos_pod = sio.loadmat(file_path_parameters)['nmodes']

# Definir o número de coeficientes a serem utilizados
num_coefs = 20  # Usando 20 coeficientes
print(f"Número total de modos utilizados: {num_coefs}")

# Separar os coeficientes em parte real e imaginária
real_coefs = np.real(coefs[:num_coefs, :])
imag_coefs = np.imag(coefs[:num_coefs, :])

# Divisão dos dados
first_interval_end = 4801
real_coefs_train = real_coefs[:, :first_interval_end:2]
imag_coefs_train = imag_coefs[:, :first_interval_end:2]
real_coefs_val = real_coefs[:, first_interval_end:]
imag_coefs_val = imag_coefs[:, first_interval_end:]

# Normalização
num_total_coefs = num_coefs * 2
scalers = []

coefs_train = np.vstack([real_coefs_train, imag_coefs_train])
coefs_val = np.vstack([real_coefs_val, imag_coefs_val])
coefs_train_normalized = np.zeros_like(coefs_train)
coefs_val_normalized = np.zeros_like(coefs_val)

for i in range(num_total_coefs):
    scaler_coef = MinMaxScaler(feature_range=(-1, 1))
    coefs_train_normalized[i, :] = scaler_coef.fit_transform(coefs_train[i, :].reshape(-1, 1)).flatten()
    coefs_val_normalized[i, :] = scaler_coef.transform(coefs_val[i, :].reshape(-1, 1)).flatten()
    scalers.append(scaler_coef)

def create_sequences(real_coefs_norm, imag_coefs_norm, seq_length):
    X, y = [], []
    num_samples = real_coefs_norm.shape[1] - seq_length
    for i in range(num_samples):
        X_seq_real = real_coefs_norm[:, i:i+seq_length]
        X_seq_imag = imag_coefs_norm[:, i:i+seq_length]
        X_seq = np.vstack([X_seq_real, X_seq_imag])
        X.append(X_seq)
        y_real = real_coefs_norm[:, i+seq_length]
        y_imag = imag_coefs_norm[:, i+seq_length]
        y_seq = np.hstack([y_real, y_imag])
        y.append(y_seq)
    X = np.array(X).transpose(0, 2, 1)
    y = np.array(y)
    return X, y


# Dicionário para armazenar os resultados
results = {}

from itertools import product

keys = hyperparams_grid.keys()
values = (hyperparams_grid[key] for key in keys)

total_combinations = np.prod([len(v) for v in hyperparams_grid.values()])
print(f"Total de combinações de hiperparâmetros: {total_combinations}")

combination_index = 1

for hp_values in product(*values):
    hyperparams = dict(zip(keys, hp_values))
    sequence_length = hyperparams['sequence_length']
    lstm_units = hyperparams['lstm_units']
    conv_filters = hyperparams['conv_filters']
    kernel_size = hyperparams['kernel_size']
    dropout_rate = hyperparams['dropout_rate']
    learning_rate = hyperparams['learning_rate']
    batch_size = hyperparams['batch_size']
    maxlags_var = hyperparams['maxlags_var']

    print(f"\nExecutando combinação {combination_index}/{total_combinations}: {hyperparams}")
    combination_index += 1

    # Criar sequências
    X_train, y_train = create_sequences(coefs_train_normalized[:num_coefs, :], coefs_train_normalized[num_coefs:, :], sequence_length)
    X_val, y_val = create_sequences(coefs_val_normalized[:num_coefs, :], coefs_val_normalized[num_coefs:, :], sequence_length)

    # Verificar se há dados suficientes
    if X_train.shape[0] == 0 or X_val.shape[0] == 0:
        print("Dados insuficientes para a sequência atual. Pulando para a próxima combinação.")
        continue

    # Definir o modelo
    lstm_input = Input(shape=(X_train.shape[1], X_train.shape[2]))
    x = lstm_input
    x = Conv1D(filters=conv_filters[0], kernel_size=kernel_size, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv1D(filters=conv_filters[1], kernel_size=kernel_size, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = Dropout(dropout_rate)(x)
    x = LSTM(lstm_units, return_sequences=False, activation='tanh')(x)
    x = Dropout(dropout_rate)(x)
    output = Dense(num_total_coefs, activation='linear')(x)
    model = Model(inputs=lstm_input, outputs=output)
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')

    # Treinar o modelo
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    history = model.fit(X_train, y_train, epochs=50, batch_size=batch_size, validation_data=(X_val, y_val),
                        callbacks=[early_stopping], verbose=0)

    # Previsões LSTM+CNN
    lstm_cnn_predictions_normalized = model.predict(X_val)
    lstm_cnn_predictions = np.zeros_like(lstm_cnn_predictions_normalized)
    y_val_original = np.zeros_like(y_val)
    for i in range(num_total_coefs):
        scaler = scalers[i]
        lstm_cnn_predictions[:, i] = scaler.inverse_transform(lstm_cnn_predictions_normalized[:, i].reshape(-1, 1)).flatten()
        y_val_original[:, i] = scaler.inverse_transform(y_val[:, i].reshape(-1, 1)).flatten()

    # Calcular resíduos
    residuals = y_val_original - lstm_cnn_predictions

    # Ajustar VAR nos resíduos
    model_var = VAR(residuals)
    results_aic = model_var.select_order(maxlags=maxlags_var)
    selected_lag = results_aic.aic if results_aic.aic is not None else maxlags_var
    model_var_fitted = model_var.fit(selected_lag)

    # Previsões VAR nos resíduos
    var_residuals_pred = model_var_fitted.fittedvalues
    var_residuals_pred = np.vstack([np.zeros((selected_lag, num_total_coefs)), var_residuals_pred])

    # Previsão final
    final_predictions = lstm_cnn_predictions[selected_lag:] + var_residuals_pred[selected_lag:]
    actual_values = y_val_original[selected_lag + sequence_length:]

    # Ajuste de dimensões
    min_length = min(final_predictions.shape[0], actual_values.shape[0])
    final_predictions = final_predictions[:min_length]
    actual_values = actual_values[:min_length]

    # Separar as partes real e imaginária
    pred_real = final_predictions[:, :num_coefs]
    pred_imag = final_predictions[:, num_coefs:]
    actual_real = actual_values[:, :num_coefs]
    actual_imag = actual_values[:, num_coefs:]

    # Cálculo do NMSE
    actual_modulus = np.sqrt(actual_real**2 + actual_imag**2)
    predicted_modulus = np.sqrt(pred_real**2 + pred_imag**2)

    nmse_modulus = []
    for c in range(num_coefs):
        mse = mean_squared_error(actual_modulus[:, c], predicted_modulus[:, c])
        variance = np.var(actual_modulus[:, c])
        nmse = mse / variance if variance != 0 else np.nan
        nmse_modulus.append(nmse)

    average_nmse_modulus = np.nanmean(nmse_modulus)

    # Salvar resultados
    result_key = f"SeqLen:{sequence_length}_Units:{lstm_units}_Filters:{conv_filters}_Kernel:{kernel_size}_Dropout:{dropout_rate}_LR:{learning_rate}_Batch:{batch_size}_VARLag:{selected_lag}"
    results[result_key] = average_nmse_modulus
    print(f"NMSE médio do módulo dos coeficientes: {average_nmse_modulus:.6f}")

# Salvar os resultados em um arquivo
import pickle
with open('grid_search_results_lstm_cnn_var.pkl', 'wb') as f:
    pickle.dump(results, f)

# Encontrar a melhor combinação de hiperparâmetros
best_params = min(results, key=results.get)
print(f"\nMelhores hiperparâmetros: {best_params} com NMSE: {results[best_params]:.6f}")


Número total de modos utilizados: 20
Total de combinações de hiperparâmetros: 96

Executando combinação 1/96: {'sequence_length': 10, 'lstm_units': 64, 'conv_filters': [32, 16], 'kernel_size': 3, 'dropout_rate': 0.1, 'learning_rate': 0.001, 'batch_size': 32, 'maxlags_var': 5}
175/175 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step
NMSE médio do módulo dos coeficientes: 1.907957

Executando combinação 2/96: {'sequence_length': 10, 'lstm_units': 64, 'conv_filters': [32, 16], 'kernel_size': 3, 'dropout_rate': 0.1, 'learning_rate': 0.001, 'batch_size': 32, 'maxlags_var': 10}
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
NMSE médio do módulo dos coeficientes: 1.908020

Executando combinação 3/96: {'sequence_length': 10, 'lstm_units': 64, 'conv_filters': [32, 16], 'kernel_size': 3, 'dropout_rate': 0.1, 'learning_rate': 0.001, 'batch_size': 64, 'maxlags_var': 5}
175/175 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step
NMSE médio do módulo dos coeficientes: 1.910074

Executando combinação 4/96: {'sequence_length': 10, 'lstm_unit